# कार्टपोल बैलेंसिंग के लिए RL प्रशिक्षण

यह नोटबुक [AI for Beginners Curriculum](http://aka.ms/ai-beginners) का हिस्सा है। इसे [आधिकारिक PyTorch ट्यूटोरियल](https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) और [इस Cartpole PyTorch कार्यान्वयन](https://github.com/yc930401/Actor-Critic-pytorch) से प्रेरित किया गया है।

इस उदाहरण में, हम RL का उपयोग करके एक मॉडल को प्रशिक्षित करेंगे ताकि वह एक पोल को संतुलित कर सके, जो एक गाड़ी पर रखा गया है और क्षैतिज पैमाने पर बाएं और दाएं चल सकता है। हम [OpenAI Gym](https://www.gymlibrary.ml/) वातावरण का उपयोग करेंगे ताकि इस पोल का अनुकरण किया जा सके।

> **Note**: आप इस पाठ का कोड स्थानीय रूप से (जैसे Visual Studio Code से) चला सकते हैं, इस स्थिति में अनुकरण एक नई विंडो में खुलेगा। जब आप कोड ऑनलाइन चला रहे हों, तो आपको कोड में कुछ बदलाव करने की आवश्यकता हो सकती है, जैसा कि [यहां](https://towardsdatascience.com/rendering-openai-gym-envs-on-binder-and-google-colab-536f99391cc7) वर्णित है।

हम शुरुआत करेंगे यह सुनिश्चित करके कि Gym इंस्टॉल है:


In [ ]:
import sys
!{sys.executable} -m pip install gym

अब आइए CartPole एनवायरनमेंट बनाते हैं और देखते हैं कि इसे कैसे संचालित किया जाए। एक एनवायरनमेंट में निम्नलिखित गुण होते हैं:

* **एक्शन स्पेस** वह सेट है जिसमें वे सभी संभावित क्रियाएं शामिल हैं जो हम सिमुलेशन के प्रत्येक चरण में कर सकते हैं।  
* **ऑब्ज़र्वेशन स्पेस** वह स्पेस है जिसमें वे सभी ऑब्ज़र्वेशन शामिल हैं जो हम कर सकते हैं।  


In [ ]:
import gym

env = gym.make("CartPole-v1")

print(f"Action space: {env.action_space}")
print(f"Observation space: {env.observation_space}")

आइए देखें कि सिमुलेशन कैसे काम करता है। निम्नलिखित लूप सिमुलेशन को तब तक चलाता है जब तक कि `env.step` समाप्ति फ्लैग `done` वापस नहीं करता। हम `env.action_space.sample()` का उपयोग करके क्रियाओं को यादृच्छिक रूप से चुनेंगे, जिसका मतलब है कि प्रयोग शायद बहुत जल्दी विफल हो जाएगा (CartPole वातावरण तब समाप्त होता है जब CartPole की गति, उसकी स्थिति या कोण कुछ सीमाओं के बाहर होते हैं)।

> सिमुलेशन एक नई विंडो में खुलेगा। आप कोड को कई बार चला सकते हैं और देख सकते हैं कि यह कैसा व्यवहार करता है।


In [ ]:
env.reset()

done = False
total_reward = 0
while not done:
   env.render()
   obs, rew, done, info = env.step(env.action_space.sample())
   total_reward += rew
   print(f"{obs} -> {rew}")
print(f"Total reward: {total_reward}")

आप देख सकते हैं कि अवलोकन में 4 संख्याएँ होती हैं। ये हैं:
- गाड़ी की स्थिति
- गाड़ी की गति
- पोल का कोण
- पोल का घूर्णन दर

`rew` वह इनाम है जो हमें प्रत्येक चरण पर मिलता है। आप देख सकते हैं कि CartPole वातावरण में आपको प्रत्येक सिमुलेशन चरण के लिए 1 अंक का इनाम मिलता है, और लक्ष्य कुल इनाम को अधिकतम करना है, यानी CartPole को गिरने के बिना संतुलन बनाए रखने का समय।

पुनर्बलन शिक्षण (Reinforcement Learning) के दौरान, हमारा लक्ष्य एक **नीति** $\pi$ को प्रशिक्षित करना है, जो प्रत्येक स्थिति $s$ के लिए हमें बताएगी कि कौन सा कार्य $a$ करना है, यानी मूल रूप से $a = \pi(s)$।

यदि आप संभाव्य समाधान चाहते हैं, तो आप नीति को प्रत्येक कार्य के लिए संभावनाओं का सेट लौटाने के रूप में सोच सकते हैं, यानी $\pi(a|s)$ का मतलब होगा कि स्थिति $s$ पर हमें कार्य $a$ करने की कितनी संभावना है।

## नीति ग्रेडिएंट विधि

सबसे सरल RL एल्गोरिदम, जिसे **नीति ग्रेडिएंट** कहा जाता है, में हम एक न्यूरल नेटवर्क को प्रशिक्षित करेंगे ताकि वह अगला कार्य भविष्यवाणी कर सके।


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch

num_inputs = 4
num_actions = 2

model = torch.nn.Sequential(
    torch.nn.Linear(num_inputs, 128, bias=False, dtype=torch.float32),
    torch.nn.ReLU(),
    torch.nn.Linear(128, num_actions, bias = False, dtype=torch.float32),
    torch.nn.Softmax(dim=1)
)

हम नेटवर्क को कई प्रयोग चलाकर प्रशिक्षित करेंगे, और प्रत्येक रन के बाद अपने नेटवर्क को अपडेट करेंगे। आइए एक फ़ंक्शन परिभाषित करें जो प्रयोग चलाएगा और परिणाम लौटाएगा (जिसे **ट्रेस** कहा जाता है) - सभी अवस्थाएँ, क्रियाएँ (और उनकी अनुशंसित संभावनाएँ), और पुरस्कार:


In [ ]:
def run_episode(max_steps_per_episode = 10000,render=False):    
    states, actions, probs, rewards = [],[],[],[]
    state = env.reset()
    for _ in range(max_steps_per_episode):
        if render:
            env.render()
        action_probs = model(torch.from_numpy(np.expand_dims(state,0)))[0]
        action = np.random.choice(num_actions, p=np.squeeze(action_probs.detach().numpy()))
        nstate, reward, done, info = env.step(action)
        if done:
            break
        states.append(state)
        actions.append(action)
        probs.append(action_probs.detach().numpy())
        rewards.append(reward)
        state = nstate
    return np.vstack(states), np.vstack(actions), np.vstack(probs), np.vstack(rewards)

आप बिना प्रशिक्षित नेटवर्क के एक एपिसोड चला सकते हैं और देख सकते हैं कि कुल इनाम (जिसे एपिसोड की लंबाई भी कहा जाता है) बहुत कम है:


In [ ]:
s, a, p, r = run_episode()
print(f"Total reward: {np.sum(r)}")

पॉलिसी ग्रेडिएंट एल्गोरिदम के जटिल पहलुओं में से एक **डिस्काउंटेड रिवार्ड्स** का उपयोग करना है। विचार यह है कि हम खेल के प्रत्येक चरण में कुल रिवार्ड्स का वेक्टर गणना करते हैं, और इस प्रक्रिया के दौरान हम प्रारंभिक रिवार्ड्स को कुछ गुणांक $gamma$ का उपयोग करके डिस्काउंट करते हैं। हम परिणामी वेक्टर को भी सामान्यीकृत करते हैं, क्योंकि हम इसे अपने प्रशिक्षण को प्रभावित करने के लिए वज़न के रूप में उपयोग करेंगे:


In [ ]:
eps = 0.0001

def discounted_rewards(rewards,gamma=0.99,normalize=True):
    ret = []
    s = 0
    for r in rewards[::-1]:
        s = r + gamma * s
        ret.insert(0, s)
    if normalize:
        ret = (ret-np.mean(ret))/(np.std(ret)+eps)
    return ret

अब हम असली प्रशिक्षण शुरू करेंगे! हम 300 एपिसोड चलाएंगे, और हर एपिसोड में निम्नलिखित कार्य करेंगे:

1. प्रयोग चलाएं और ट्रेस एकत्र करें।
1. लिए गए कार्यों और अनुमानित संभावनाओं के बीच का अंतर (`gradients`) गणना करें। अंतर जितना कम होगा, उतना ही अधिक हमें यकीन होगा कि हमने सही कार्य लिया है।
1. छूटे हुए पुरस्कारों की गणना करें और उन्हें `gradients` के साथ गुणा करें - यह सुनिश्चित करेगा कि उच्च पुरस्कार वाले कदम अंतिम परिणाम पर कम पुरस्कार वाले कदमों की तुलना में अधिक प्रभाव डालें।
1. हमारे न्यूरल नेटवर्क के लिए अपेक्षित लक्ष्य कार्य आंशिक रूप से रन के दौरान अनुमानित संभावनाओं से और आंशिक रूप से गणना किए गए `gradients` से लिए जाएंगे। हम `alpha` पैरामीटर का उपयोग करेंगे यह निर्धारित करने के लिए कि `gradients` और पुरस्कारों को किस हद तक ध्यान में रखा जाए - इसे *रीइन्फोर्समेंट एल्गोरिदम का लर्निंग रेट* कहा जाता है।
1. अंत में, हम अपने नेटवर्क को स्टेट्स और अपेक्षित कार्यों पर प्रशिक्षित करेंगे, और इस प्रक्रिया को दोहराएंगे।


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_on_batch(x, y):
    x = torch.from_numpy(x)
    y = torch.from_numpy(y)
    optimizer.zero_grad()
    predictions = model(x)
    loss = -torch.mean(torch.log(predictions) * y)
    loss.backward()
    optimizer.step()
    return loss

In [ ]:
alpha = 1e-4

history = []
for epoch in range(300):
    states, actions, probs, rewards = run_episode()
    one_hot_actions = np.eye(2)[actions.T][0]
    gradients = one_hot_actions-probs
    dr = discounted_rewards(rewards)
    gradients *= dr
    target = alpha*np.vstack([gradients])+probs
    train_on_batch(states,target)
    history.append(np.sum(rewards))
    if epoch%100==0:
        print(f"{epoch} -> {np.sum(rewards)}")

plt.plot(history)

अब चलिए एपिसोड को रेंडरिंग के साथ चलाते हैं ताकि परिणाम देख सकें:


In [ ]:
_ = run_episode(render=True)

उम्मीद है कि अब आप देख सकते हैं कि पोल अब काफी अच्छी तरह से संतुलन बना सकता है!

## एक्टर-क्रिटिक मॉडल

एक्टर-क्रिटिक मॉडल पॉलिसी ग्रेडिएंट्स का एक और विकास है, जिसमें हम एक न्यूरल नेटवर्क बनाते हैं जो पॉलिसी और अनुमानित रिवॉर्ड्स दोनों को सीखता है। इस नेटवर्क के दो आउटपुट होंगे (या आप इसे दो अलग-अलग नेटवर्क के रूप में देख सकते हैं):
* **एक्टर** हमें स्टेट प्रॉबेबिलिटी डिस्ट्रीब्यूशन देकर कार्रवाई की सिफारिश करेगा, जैसा कि पॉलिसी ग्रेडिएंट मॉडल में होता है।
* **क्रिटिक** यह अनुमान लगाएगा कि उन कार्रवाइयों से रिवॉर्ड क्या होगा। यह दिए गए स्टेट पर भविष्य में कुल अनुमानित रिवॉर्ड्स लौटाता है।

आइए ऐसा एक मॉडल परिभाषित करें:


In [ ]:
from itertools import count
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
env = gym.make("CartPole-v1")

state_size = env.observation_space.shape[0]
action_size = env.action_space.n
lr = 0.0001

class Actor(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, self.action_size)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        output = self.linear3(output)
        distribution = torch.distributions.Categorical(F.softmax(output, dim=-1))
        return distribution


class Critic(torch.nn.Module):
    def __init__(self, state_size, action_size):
        super(Critic, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.linear1 = torch.nn.Linear(self.state_size, 128)
        self.linear2 = torch.nn.Linear(128, 256)
        self.linear3 = torch.nn.Linear(256, 1)

    def forward(self, state):
        output = F.relu(self.linear1(state))
        output = F.relu(self.linear2(output))
        value = self.linear3(output)
        return value

हमें अपनी `discounted_rewards` और `run_episode` फ़ंक्शनों को थोड़ा संशोधित करने की आवश्यकता होगी:


In [ ]:
def discounted_rewards(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def run_episode(actor, critic, n_iters):
    optimizerA = torch.optim.Adam(actor.parameters())
    optimizerC = torch.optim.Adam(critic.parameters())
    for iter in range(n_iters):
        state = env.reset()
        log_probs = []
        values = []
        rewards = []
        masks = []
        entropy = 0
        env.reset()

        for i in count():
            env.render()
            state = torch.FloatTensor(state).to(device)
            dist, value = actor(state), critic(state)

            action = dist.sample()
            next_state, reward, done, _ = env.step(action.cpu().numpy())

            log_prob = dist.log_prob(action).unsqueeze(0)
            entropy += dist.entropy().mean()

            log_probs.append(log_prob)
            values.append(value)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=device))
            masks.append(torch.tensor([1-done], dtype=torch.float, device=device))

            state = next_state

            if done:
                print('Iteration: {}, Score: {}'.format(iter, i))
                break


        next_state = torch.FloatTensor(next_state).to(device)
        next_value = critic(next_state)
        returns = discounted_rewards(next_value, rewards, masks)

        log_probs = torch.cat(log_probs)
        returns = torch.cat(returns).detach()
        values = torch.cat(values)

        advantage = returns - values

        actor_loss = -(log_probs * advantage.detach()).mean()
        critic_loss = advantage.pow(2).mean()

        optimizerA.zero_grad()
        optimizerC.zero_grad()
        actor_loss.backward()
        critic_loss.backward()
        optimizerA.step()
        optimizerC.step()


अब हम मुख्य प्रशिक्षण लूप चलाएंगे। हम उचित हानि फ़ंक्शन की गणना करके और नेटवर्क पैरामीटर को अपडेट करके मैनुअल नेटवर्क प्रशिक्षण प्रक्रिया का उपयोग करेंगे:


In [ ]:

actor = Actor(state_size, action_size).to(device)
critic = Critic(state_size, action_size).to(device)
run_episode(actor, critic, n_iters=100)

In [ ]:
env.close()

## मुख्य बातें

हमने इस डेमो में दो RL एल्गोरिदम देखे: सरल पॉलिसी ग्रेडिएंट और अधिक परिष्कृत एक्टर-क्रिटिक। आप देख सकते हैं कि ये एल्गोरिदम स्टेट, एक्शन और रिवॉर्ड जैसे अमूर्त विचारों के साथ काम करते हैं - इसलिए इन्हें बहुत अलग-अलग वातावरणों में लागू किया जा सकता है।

रीइन्फोर्समेंट लर्निंग हमें केवल अंतिम रिवॉर्ड को देखकर समस्या को हल करने की सबसे अच्छी रणनीति सीखने की अनुमति देता है। यह तथ्य कि हमें लेबल किए गए डेटा सेट की आवश्यकता नहीं होती, हमें अपने मॉडल को अनुकूलित करने के लिए सिमुलेशन को कई बार दोहराने की सुविधा देता है। हालांकि, RL में अभी भी कई चुनौतियां हैं, जिन्हें आप सीख सकते हैं यदि आप AI के इस रोचक क्षेत्र पर अधिक ध्यान केंद्रित करने का निर्णय लेते हैं।



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता के लिए प्रयासरत हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
